# 🎨 WIWS ComfyUI Colab (Ultimate Edition)

**Created by WIWS_MPA_C00_AI_DNA_Core_Autonomous_System**

이 노트는 최신 **ComfyUI**를 Google Colab에서 가장 안정적이고 빠르게 실행할 수 있도록 설계되었습니다.
기존 WebUI보다 가볍고, 오류가 없으며, 최신 AI 기술을 모두 지원합니다.

### 1. 구글 드라이브 연동 (필수)
기존에 사용하던 모델 폴더(`WIWS_SD_Forge`)를 그대로 재활용하거나, 없으면 새로 만듭니다.

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# 기존 폴더 재활용 (이름이 달라도 같이 쓸 수 있게 설정)
base_path = '/content/drive/MyDrive/WIWS_SD_Forge'
if not os.path.exists(base_path):
    base_path = '/content/drive/MyDrive/WIWS_ComfyUI'
    os.makedirs(base_path, exist_ok=True)

os.makedirs(f'{base_path}/outputs', exist_ok=True)
os.makedirs(f'{base_path}/models', exist_ok=True)
os.makedirs(f'{base_path}/models/checkpoints', exist_ok=True)
os.makedirs(f'{base_path}/models/loras', exist_ok=True)
os.makedirs(f'{base_path}/models/vae', exist_ok=True)
os.makedirs(f'{base_path}/models/controlnet', exist_ok=True)

print(f"✅ 구글 드라이브 연결 완료! 저장 경로: {base_path}")

### 2. ComfyUI 설치 및 실행
약 3~5분 소요됩니다. 설치가 완료되면 **`trycloudflare.com`** 으로 끝나는 링크가 나옵니다.
그 링크를 클릭하면 ComfyUI가 열립니다.

In [ ]:
import os

%cd /content

# 1. ComfyUI 클론
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI

# 2. ComfyUI Manager 설치 (필수 확장기능)
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd /content/ComfyUI

# 3. 구글 드라이브 모델 연동 (심볼릭 링크)
# 기존 WebUI 폴더 구조와 ComfyUI 구조를 매핑합니다.
!rm -rf /content/ComfyUI/models/checkpoints
!ln -s "{base_path}/models" /content/ComfyUI/models/checkpoints
!rm -rf /content/ComfyUI/output
!ln -s "{base_path}/outputs" /content/ComfyUI/output

# 4. 필수 패키지 설치
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121

# 5. Cloudflared 터널링 설치 (안정적인 접속)
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# 6. 실행
import subprocess
import threading
import time
import socket

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here, please restart the colab)...")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("\n\033[92m🔥 접속 링크 (클릭하세요): " + l[l.find("http"):], end="\033[0m\n")

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### 💡 사용 팁
1. **ComfyUI Manager**: 화면 우측 하단 'Manager' 버튼을 통해 모델, 노드를 쉽게 설치할 수 있습니다.
2. **모델 넣기**: 구글 드라이브 `WIWS_SD_Forge/models` 폴더에 `.safetensors` 파일을 넣고 'Refresh' 하세요.
3. **기본 사용**: 처음 실행 시 기본 워크플로우가 뜹니다. `Queue Prompt` 버튼을 누르면 이미지가 생성됩니다.